In [5]:
import os
import torch
import pandas as pd
import numpy as np
from transformers import RobertaForMaskedLM
sys.path.append('/home/amber/multitask_RNA/DNA_BERT_rep/')
from dna_tokenizer import DNATokenizer
os.environ['CUDA_VISIBLE_DEVICES']= '7'
def onehotcoding(seq):
    IN_MAP = np.asarray([[0, 0, 0, 0],
                     [1, 0, 0, 0],
                     [0, 1, 0, 0],
                     [0, 0, 1, 0],
                     [0, 0, 0, 1]])
    seq = seq.upper().replace('A', '1').replace('C', '2')
    seq = seq.replace('G', '3').replace('T', '4').replace('N', '0')
    return IN_MAP[np.array(list(seq)).astype(int)]

In [6]:
model = RobertaForMaskedLM.from_pretrained('/home/amber/multitask_RNA/DNA_BERT_rep/small-roberta-lr8/checkpoint-23500/')
tokenizer = DNATokenizer('/home/amber/multitask_RNA/DNA_BERT_rep/vocab.txt')
rna_local = pd.read_csv('../data/RNA_loc/GSE173098_RNAloc_MPRA_counts.csv',index_col = 0)

In [ ]:
device=torch.device('cuda:0')
model = model.to(device)

In [3]:
x = []
y = []
for (i,row) in rna_local.iterrows():
    seq = row['synthetic library sequence']
    vector = row.values[1:]
    
    seq_token = tokenizer.batch_encode_plus(seq,return_tensors='pt',
                                    add_special_tokens=True)
    x.append(seq)
    y.append(np.array(vector).astype(int))

KeyboardInterrupt: 

In [53]:
x = []
y = []
for (i,row) in rna_local.iterrows():
    seq = row['synthetic library sequence']
    vector = row.values[1:]
    onehot = onehotcoding(seq)
    x.append(onehot)
    y.append(np.array(vector).astype(int))

In [48]:
import h5py
file_name = './data/neuron_local/RNAloc.h5'
h5f = h5py.File(file_name, 'w')
x = h5f.create_dataset('x',data = np.array(x))
y = h5f.create_dataset('y',data =np.array(y))

TypeError: Object dtype dtype('O') has no native HDF5 equivalent